In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import os.path as osp
import itertools
import copy
import uuid
import pickle


import torch
import torch.nn.functional as F
import torch_geometric
import numpy as np

import torch.nn as nn
import torch.optim as optim

import chofer_torchex.pershom as pershom

from torch.nn import Sequential, Linear, ReLU
from torch.optim.lr_scheduler import MultiStepLR

from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GINConv, global_add_pool

from chofer_torchex import pershom
ph = pershom.pershom_backend.__C.VertFiltCompCuda__vert_filt_persistence_batch

from chofer_torchex.nn import SLayerRationalHat
from collections import defaultdict, Counter

device = 'cuda:1'

import core.model
from core.data import dataset_factory, train_test_split
from core.utils import my_collate, evaluate
from core.train_engine import *

from tensorboardX import SummaryWriter



In [28]:
dataset_names = [
        #'COLLAB',
        'IMDB-MULTI',
        'IMDB-BINARY',
        'ENZYMES',
        #'PTC_PGNN',
        #'PTC_FM',
        #'PTC_FR',
        #'PTC_MM',
        #'PTC_MR',
        'PROTEINS',
        'DD',
        'NCI1',
        #'MUTAG'
]
print(dataset_names)


dataset_has_node_lab = {n: dataset_factory(n, verbose=False).num_node_lab is not None for n in dataset_names}
dataset_has_node_lab

training_cfg = {
    'lr': 0.01, 
    'lr_drop_fact': 0.5, 
    'num_epochs': 80,
    'epoch_step': 20,
    'batch_size': 64,
    'weight_decay': 10e-06,
}
training_cfgs = [training_cfg]

model_cfg = {
    'model_type': 'PershomModel',
    'model_kwargs': {
        'use_sup_lvlset_filt': None,
        'filtration_kwargs': {
            'use_node_deg': None,
            'use_node_lab': None,
            'num_gin': 1,
            'hidden_dim': 64, 
            'use_mlp': None
        }, 
        'classifier_kwargs': {
            'num_struct_elem': 50
        }
    }
}

filt_cfgs = []
for a, b, c in itertools.product([True, False], [True, False], [True]):
    if (not a) and (not b):
        continue

    tmp = copy.deepcopy(model_cfg)
    tmp['model_kwargs']['filtration_kwargs']['use_node_deg'] = a
    tmp['model_kwargs']['filtration_kwargs']['use_node_lab'] = b
    tmp['model_kwargs']['filtration_kwargs']['use_mlp'] = c
    

    filt_cfgs.append(tmp)

    
no_filt_learning = copy.deepcopy(model_cfg)
no_filt_learning['model_kwargs']['filtration_kwargs'] = None
filt_cfgs.append(no_filt_learning)

model_cfgs = []
for cfg in filt_cfgs:
    for b in [True, False]:
        tmp = copy.deepcopy(cfg)
        tmp['model_kwargs']['use_sup_lvlset_filt'] = b
        model_cfgs.append(tmp)

exp_cfgs = []
for a, b, c in itertools.product(dataset_names, training_cfgs, model_cfgs):

    # filter out datasets which have no node labels
    if not dataset_has_node_lab[a]:
        if c['model_kwargs']['filtration_kwargs'] is not None:
            use_node_lab = c['model_kwargs']['filtration_kwargs']['use_node_lab']

            if use_node_lab:
                continue

    tmp = {
        'dataset_name': a, 
        'training': b, 
        'model': c
    }
    exp_cfgs.append(tmp)

print("# Number of configurations", len(exp_cfgs))

['IMDB-MULTI', 'IMDB-BINARY', 'ENZYMES', 'PROTEINS', 'DD', 'NCI1']
# Number of configurations 40


In [30]:
print(len(dataset_names), len(training_cfgs), len(model_cfgs))

6 1 8


In [23]:
for cfg in exp_cfgs:
    print(cfg['model']['model_kwargs']['filtration_kwargs']['use_node_deg'], cfg['model']['model_kwargs']['filtration_kwargs']['use_node_lab'] )

KeyError: 'model_kwargs'

In [31]:
output_dir = '/home/pma/chofer/repositories/nips_2019_code/results'
with open(os.path.join(output_dir, 'exp_cfgs.json'), 'w') as fid:
    json.dump(exp_cfgs, fid)